In [1]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier

In [2]:
hr_train = pd.read_csv('../data/hr_tr.csv')
hw_train = pd.read_csv('../data/hw_tr.csv')
hk_train = pd.read_csv('../data/hk_tr.csv')

In [4]:
hr_valid = pd.read_csv('../data/hr_val.csv')
hr_test = pd.read_csv('../data/hr_te.csv')

hw_valid = pd.read_csv('../data/hw_val.csv')
hw_test = pd.read_csv('../data/hw_te.csv')

hk_valid = pd.read_csv('../data/hk_val.csv')
hk_test = pd.read_csv('../data/hk_te.csv')

## 합치기 전 Datastep
1. 각 데이터 셋 별 group 변수 만들기
2. Biopsy 변수 target으로 변환

##### train set

In [12]:
# hr
hr_train['group'] = 'hr'
hr_train['target'] = hr_train['Biopsy_HR'].copy()
hr_train.iloc[:5,22:]

,esterase_binary,Biopsy_HR,group,target
0,1,1,hr,1
1,0,0,hr,0
2,0,0,hr,0
3,1,0,hr,0
4,1,0,hr,0


In [14]:
# hw
hw_train['group'] = 'hw'
hw_train['target'] = hw_train['Biopsy_HW'].copy()
hw_train.iloc[:5,22:]

,esterase_binary,Biopsy_HW,group,target
0,1,1,hw,1
1,0,0,hw,0
2,0,0,hw,0
3,1,0,hw,0
4,1,0,hw,0


In [15]:
# hk
hk_train['group'] = 'hk'
hk_train['target'] = hk_train['Biopsy_HK'].copy()
hk_train.iloc[:5,22:]

,esterase_binary,Biopsy_HK,group,target
0,1,1,hk,1
1,0,0,hk,0
2,0,0,hk,0
3,1,0,hk,0
4,1,1,hk,1


In [ ]:
# 변수명 다 같음 굿 잘 변환 됐음
print(list(set(hr_train.columns)^set(hk_train.columns)))
print('-'*10)
print(list(set(hr_train.columns)^set(hw_train.columns)))
print('-'*10)
print(list(set(hk_train.columns)^set(hw_train.columns)))

['Biopsy_HK', 'Biopsy_HR']
----------
['Biopsy_HW', 'Biopsy_HR']
----------
['Biopsy_HK', 'Biopsy_HW']


In [ ]:
# Biopsy 없애기
hr_train_copy = hr_train.copy()
hw_train_copy = hw_train.copy()
hk_train_copy = hk_train.copy()

hr_train_copy.drop(['Biopsy_HR'], axis=1, inplace=True)
hw_train_copy.drop(['Biopsy_HW'], axis=1, inplace=True)
hk_train_copy.drop(['Biopsy_HK'], axis=1, inplace=True)

print(hr_train_copy.iloc[:5,22:])
print(hw_train_copy.iloc[:5,22:])
print(hk_train_copy.iloc[:5,22:])

   esterase_binary group  target
0                1    hr       1
1                0    hr       0
2                0    hr       0
3                1    hr       0
4                1    hr       0
   esterase_binary group  target
0                1    hw       1
1                0    hw       0
2                0    hw       0
3                1    hw       0
4                1    hw       0
   esterase_binary group  target
0                1    hk       1
1                0    hk       0
2                0    hk       0
3                1    hk       0
4                1    hk       1


In [33]:
train_concat = pd.concat([hr_train_copy, hw_train_copy, hk_train_copy], ignore_index=True)
print(train_concat[6165:6175])
print(train_concat[12340:12350])

            age  sex  HTN  DM    plt  tpro  alb    bun    cr        eGFR  ...  \
6165  38.000000    2    0   0  252.0   4.7  2.3    8.0  0.61  115.000000  ...   
6166  55.000000    2    0   0  338.0   6.0  3.6   12.5  0.57  105.000000  ...   
6167  25.000000    2    0   0  227.0   4.5  2.1  119.0  8.56    6.000000  ...   
6168  54.000000    1    1   0  258.0   6.8  4.4   11.5  0.98   88.000000  ...   
6169  46.000000    2    0   0  300.0   6.6  4.2   17.3  0.89   78.000000  ...   
6170  58.000000    1    0   1  262.0   5.3  2.9   15.1  1.30   60.000000  ...   
6171  48.769863    2    0   0  169.0   6.1  3.5   24.3  1.84   31.676774  ...   
6172  37.068493    2    0   0  240.0   7.7  5.0   14.1  0.74  103.443334  ...   
6173  43.323288    2    0   0  337.0   6.8  4.4    8.4  0.60  111.384254  ...   
6174  71.479452    2    0   0  326.0   6.5  3.7   23.7  1.66   30.585367  ...   

      glu_urine  esterase_urine  dysmorphism  RBC_re  WBC_re  RBC_binary  \
6165        0.0             0.0 

##### valid

In [16]:
# hr
hr_valid['group'] = 'hr'
hr_valid['target'] = hr_valid['Biopsy_HR'].copy()
hr_valid.iloc[:5,22:]

,esterase_binary,Biopsy_HR,group,target
0,1,0,hr,0
1,0,0,hr,0
2,1,0,hr,0
3,0,1,hr,1
4,1,0,hr,0


In [17]:
# hw
hw_valid['group'] = 'hw'
hw_valid['target'] = hw_valid['Biopsy_HW'].copy()
hw_valid.iloc[:5,22:]

,esterase_binary,Biopsy_HW,group,target
0,1,0,hw,0
1,0,0,hw,0
2,1,0,hw,0
3,0,0,hw,0
4,1,0,hw,0


In [18]:
# hk
hk_valid['group'] = 'hk'
hk_valid['target'] = hk_valid['Biopsy_HK'].copy()
hk_valid.iloc[:5,22:]

,esterase_binary,Biopsy_HK,group,target
0,1,0,hk,0
1,0,0,hk,0
2,1,0,hk,0
3,0,1,hk,1
4,1,0,hk,0


In [28]:
# Biopsy 없애기
hr_valid_copy = hr_valid.copy()
hw_valid_copy = hw_valid.copy()
hk_valid_copy = hk_valid.copy()

hr_valid_copy.drop(['Biopsy_HR'], axis=1, inplace=True)
hw_valid_copy.drop(['Biopsy_HW'], axis=1, inplace=True)
hk_valid_copy.drop(['Biopsy_HK'], axis=1, inplace=True)

print(hr_valid_copy.iloc[:5,22:])
print(hw_valid_copy.iloc[:5,22:])
print(hk_valid_copy.iloc[:5,22:])

   esterase_binary group  target
0                1    hr       0
1                0    hr       0
2                1    hr       0
3                0    hr       1
4                1    hr       0
   esterase_binary group  target
0                1    hw       0
1                0    hw       0
2                1    hw       0
3                0    hw       0
4                1    hw       0
   esterase_binary group  target
0                1    hk       0
1                0    hk       0
2                1    hk       0
3                0    hk       1
4                1    hk       0


In [38]:
hw_valid.tail

<bound method NDFrame.tail of            age  sex  HTN  DM    plt  tpro  alb   bun    cr        eGFR  ...  \
0    54.545205    1    1   0  286.0   7.3  4.5  10.4  0.93   92.384381  ...   
1    71.783562    2    0   0  239.0   6.9  4.2  39.3  1.97   24.813438  ...   
2    72.473973    2    1   0  329.0   8.0  4.4  13.9  1.07   51.649409  ...   
3    32.328767    2    0   0  192.0   5.9  3.8  39.9  4.06   13.657003  ...   
4    64.715068    2    1   1  226.0   6.9  4.0  24.7  2.25   22.206094  ...   
..         ...  ...  ...  ..    ...   ...  ...   ...   ...         ...  ...   
406  68.334247    1    0   0  229.0   4.1  1.7  15.2  1.01   75.893217  ...   
407  43.304110    2    1   0  305.0   5.5  3.0  21.4  0.67  107.427497  ...   
408  75.909589    1    0   0  300.0   6.6  3.9  45.1  2.30   26.606519  ...   
409  52.421918    2    0   0  243.0   7.1  4.0  11.3  0.44  115.712951  ...   
410  52.353425    2    1   0  365.0   6.8  4.2  14.5  0.68  100.320901  ...   

     esterase_urine  

In [37]:
valid_concat = pd.concat([hr_valid_copy, hw_valid_copy, hk_valid_copy], ignore_index=True)
print(valid_concat[409:414])
print(valid_concat[820:825])

           age  sex  HTN  DM    plt  tpro  alb   bun    cr        eGFR  ...  \
409  52.421918    2    0   0  243.0   7.1  4.0  11.3  0.44  115.712951  ...   
410  52.353425    2    1   0  365.0   6.8  4.2  14.5  0.68  100.320901  ...   
411  54.545205    1    1   0  286.0   7.3  4.5  10.4  0.93   92.384381  ...   
412  71.783562    2    0   0  239.0   6.9  4.2  39.3  1.97   24.813438  ...   
413  72.473973    2    1   0  329.0   8.0  4.4  13.9  1.07   51.649409  ...   

     glu_urine  esterase_urine  dysmorphism  RBC_re  WBC_re  RBC_binary  \
409        0.0             0.0          NaN    24.8    11.6           0   
410        0.0             0.0          NaN    35.6     3.5           0   
411        0.0             0.5         26.0    56.2     2.9           0   
412        0.0             2.0          NaN    24.4    64.7           0   
413        0.0             0.5          NaN    13.3     7.9           0   

     WBC_binary  esterase_binary  group  target  
409           0         

##### test

In [19]:
# hr
hr_test['group'] = 'hr'
hr_test['target'] = hr_test['Biopsy_HR'].copy()
hr_test.iloc[:5,22:]

,esterase_binary,Biopsy_HR,group,target
0,1,0,hr,0
1,1,0,hr,0
2,1,0,hr,0
3,1,0,hr,0
4,1,0,hr,0


In [20]:
# hw
hw_test['group'] = 'hw'
hw_test['target'] = hw_test['Biopsy_HW'].copy()
hw_test.iloc[:5,22:]

,esterase_binary,Biopsy_HW,group,target
0,1,0,hw,0
1,1,0,hw,0
2,1,0,hw,0
3,1,0,hw,0
4,1,0,hw,0


In [21]:
# hk
hk_test['group'] = 'hk'
hk_test['target'] = hk_test['Biopsy_HK'].copy()
hk_test.iloc[:5,22:]

,esterase_binary,Biopsy_HK,group,target
0,1,0,hk,0
1,1,1,hk,1
2,1,0,hk,0
3,1,0,hk,0
4,1,0,hk,0


In [29]:
# Biopsy 없애기
hr_test_copy = hr_test.copy()
hw_test_copy = hw_test.copy()
hk_test_copy = hk_test.copy()

hr_test_copy.drop(['Biopsy_HR'], axis=1, inplace=True)
hw_test_copy.drop(['Biopsy_HW'], axis=1, inplace=True)
hk_test_copy.drop(['Biopsy_HK'], axis=1, inplace=True)

print(hr_test_copy.iloc[:5,22:])
print(hw_test_copy.iloc[:5,22:])
print(hk_test_copy.iloc[:5,22:])

   esterase_binary group  target
0                1    hr       0
1                1    hr       0
2                1    hr       0
3                1    hr       0
4                1    hr       0
   esterase_binary group  target
0                1    hw       0
1                1    hw       0
2                1    hw       0
3                1    hw       0
4                1    hw       0
   esterase_binary group  target
0                1    hk       0
1                1    hk       1
2                1    hk       0
3                1    hk       0
4                1    hk       0


In [41]:
test_concat = pd.concat([hr_test_copy, hw_test_copy, hk_test_copy], ignore_index=True)
print(test_concat[1643:1648])
print(test_concat[3290:3295])

            age  sex  HTN  DM    plt  tpro  alb   bun    cr        eGFR  ...  \
1643  33.558904    1    0   0  222.0   7.4  4.5  14.8  0.86  113.489072  ...   
1644  69.509589    2    1   0  210.0   7.1  4.1  50.8  4.58    9.091752  ...   
1645  45.843836    1    1   0  297.0   8.0  4.8  14.7  1.20   72.162244  ...   
1646  79.810959    2    0   1  295.0   6.0  3.0  44.3  2.40   18.472753  ...   
1647  21.197260    1    0   0  311.0   7.5  4.8  13.4  0.89  122.052286  ...   

      glu_urine  esterase_urine  dysmorphism  RBC_re  WBC_re  RBC_binary  \
1643        0.0             0.0          NaN    10.0    10.0           0   
1644        0.0             1.0          NaN    10.0     NaN           0   
1645        0.0             0.0          NaN    10.0    10.0           0   
1646        3.0             0.0          NaN     5.8   162.0           1   
1647        0.0             0.5         20.0  6915.1    11.4           0   

      WBC_binary  esterase_binary  group  target  
1643       

### 내보내기

In [42]:
train_concat.to_csv('../data/train_concat.csv', index=False)
valid_concat.to_csv('../data/valid_concat.csv', index=False)
test_concat.to_csv('../data/test_concat.csv', index=False)